# 参数管理

我们首先关注具有单隐藏层的多层感知机

In [ ]:
import sys
sys.path.append('..')

In [1]:
from d2l import mindspore as d2l
from mindspore import nn

net = nn.SequentialCell([nn.Dense(4, 8), nn.ReLU(), nn.Dense(8, 1)])
X = d2l.rand((2, 4))
net(X)

Tensor(shape=[2, 1], dtype=Float32, value=
[[-3.49030714e-04],
 [-5.60830755e-04]])

参数访问

In [2]:
print(net[2].parameters_dict())

OrderedDict([('2.weight', Parameter (name=2.weight, shape=(1, 8), dtype=Float32, requires_grad=True)), ('2.bias', Parameter (name=2.bias, shape=(1,), dtype=Float32, requires_grad=True))])


目标参数

In [3]:
print(type(net[2].bias))
print(net[2].bias)
print(net[2].bias.value())

<class 'mindspore.common.parameter.ParameterTensor'>
Parameter (name=2.bias, shape=(1,), dtype=Float32, requires_grad=True)
Parameter (name=2.bias, shape=(1,), dtype=Float32, requires_grad=True)


一次性访问所有参数

In [4]:
print(*[(name, param.shape) for name, param in net[0].parameters_dict().items()])
print(*[(name, param.shape) for name, param in net.parameters_dict().items()])

('0.weight', (8, 4)) ('0.bias', (8,))
('0.weight', (8, 4)) ('0.bias', (8,)) ('2.weight', (1, 8)) ('2.bias', (1,))


In [5]:
net.parameters_dict()['2.bias'].value()

Parameter (name=2.bias, shape=(1,), dtype=Float32, requires_grad=True)

从嵌套块收集参数

In [6]:
def block1():
    return nn.SequentialCell([nn.Dense(4, 8), nn.ReLU(),
                              nn.Dense(8, 4), nn.ReLU()])

def block2():
    net = nn.SequentialCell()
    for i in range(4):
        # 在这里嵌套
        net.append(block1())
    return net

rgnet = nn.SequentialCell([block2(), nn.Dense(4, 1)])
rgnet(X)

Tensor(shape=[2, 1], dtype=Float32, value=
[[-3.91725408e-17],
 [-2.82850341e-17]])

我们已经设计了网络，让我们看看它是如何组织的

In [7]:
print(rgnet)

SequentialCell<
  (0): SequentialCell<
    (0): SequentialCell<
      (0): Dense<input_channels=4, output_channels=8, has_bias=True>
      (1): ReLU<>
      (2): Dense<input_channels=8, output_channels=4, has_bias=True>
      (3): ReLU<>
      >
    (1): SequentialCell<
      (0): Dense<input_channels=4, output_channels=8, has_bias=True>
      (1): ReLU<>
      (2): Dense<input_channels=8, output_channels=4, has_bias=True>
      (3): ReLU<>
      >
    (2): SequentialCell<
      (0): Dense<input_channels=4, output_channels=8, has_bias=True>
      (1): ReLU<>
      (2): Dense<input_channels=8, output_channels=4, has_bias=True>
      (3): ReLU<>
      >
    (3): SequentialCell<
      (0): Dense<input_channels=4, output_channels=8, has_bias=True>
      (1): ReLU<>
      (2): Dense<input_channels=8, output_channels=4, has_bias=True>
      (3): ReLU<>
      >
    >
  (1): Dense<input_channels=4, output_channels=1, has_bias=True>
  >


In [8]:
rgnet[0][1][0].bias.value()

Parameter (name=0.1.0.bias, shape=(8,), dtype=Float32, requires_grad=True)

默认情况下，MindSpore会使用Normal初始化权重矩阵，
偏置参数设置为0。
MindSpore的`common.initializer`模块中提供了各种初始化方法。

内置初始化

In [9]:
net = nn.SequentialCell([nn.Dense(4, 8, weight_init='normal', bias_init='zero'),
                         nn.ReLU(),
                         nn.Dense(8, 1, weight_init='normal', bias_init='zero')])

net[0].weight.data[0], net[0].bias.data[0]

(Tensor(shape=[4], dtype=Float32, value= [-1.20215854e-02, -1.25061749e-02, -7.52139417e-03,  1.09155132e-02]),
 Tensor(shape=[], dtype=Float32, value= 0))

In [10]:
net = nn.SequentialCell([nn.Dense(4, 8, weight_init='one', bias_init='zero'),
                         nn.ReLU(),
                         nn.Dense(8, 1, weight_init='one', bias_init='zero')])

net[0].weight.data[0], net[0].bias.data[0]

(Tensor(shape=[4], dtype=Float32, value= [ 1.00000000e+00,  1.00000000e+00,  1.00000000e+00,  1.00000000e+00]),
 Tensor(shape=[], dtype=Float32, value= 0))

对某些块应用不同的初始化方法

In [11]:
net = nn.SequentialCell([nn.Dense(4, 8, weight_init='xavier_uniform'),
                         nn.ReLU(),
                         nn.Dense(8, 1, weight_init=42)])

print(net[0].weight.data[0])
print(net[2].weight.data[0])

[0.23717844 0.39029706 0.04387231 0.58519274]
[42. 42. 42. 42. 42. 42. 42. 42.]


自定义初始化
同样，我们实现了一个`my_init`函数来应用到`net`。

In [12]:
def my_init(shape):
    weight = d2l.uniform(shape, -10, 10)
    weight *= d2l.abs(weight) >= 5
    return weight


net = nn.SequentialCell([nn.Dense(4, 8, weight_init=my_init((8, 4))),
                         nn.ReLU(),
                         nn.Dense(8, 1, weight_init=my_init((1, 8)))])
net[0].weight[:2]

Tensor(shape=[2, 4], dtype=Float32, value=
[[ 5.15057087e+00, -0.00000000e+00,  6.03760958e+00, -8.03363037e+00],
 [-5.74584198e+00,  0.00000000e+00, -0.00000000e+00,  7.52724266e+00]])

In [13]:
net[0].weight.data[:] += 1
net[0].weight.data[0, 0] = 42
net[0].weight.data[0]

[WARNING] KERNEL(4095486,7fc4402bd740,python):2021-11-07-23:27:27.742.999 [mindspore/ccsrc/backend/kernel_compiler/gpu/gpu_kernel_factory.cc:96] ReducePrecision] Kernel [TensorScatterUpdate] does not support int64, cast input 1 to int32.
[WARNING] PRE_ACT(4095486,7fc4402bd740,python):2021-11-07-23:27:27.743.111 [mindspore/ccsrc/backend/optimizer/gpu/reduce_precision_fusion.cc:83] Run] Reduce precision for [TensorScatterUpdate] input 1


Tensor(shape=[4], dtype=Float32, value= [ 4.20000000e+01,  1.00000000e+00,  7.03760958e+00, -7.03363037e+00])

参数绑定

In [14]:
# 我们需要给共享层一个名称，以便可以引用它的参数
shared = nn.Dense(8, 8)
net = nn.SequentialCell([nn.Dense(4, 8),
                         nn.ReLU(),
                         shared,
                         nn.ReLU(),
                         shared,
                         nn.ReLU(),
                         nn.Dense(8, 1)])
net(X)
# 检查参数是否相同
print(net[2].weight.data[0] == net[4].weight.data[0])
net[2].weight.data[0, 0] = 100
# 确保它们实际上是同一个对象，而不只是有相同的值
print(net[2].weight.data[0] == net[4].weight.data[0])

[ True  True  True  True  True  True  True  True]
[ True  True  True  True  True  True  True  True]


[WARNING] KERNEL(4095486,7fc4402bd740,python):2021-11-07-23:27:27.830.726 [mindspore/ccsrc/backend/kernel_compiler/gpu/gpu_kernel_factory.cc:96] ReducePrecision] Kernel [TensorScatterUpdate] does not support int64, cast input 1 to int32.
[WARNING] PRE_ACT(4095486,7fc4402bd740,python):2021-11-07-23:27:27.830.784 [mindspore/ccsrc/backend/optimizer/gpu/reduce_precision_fusion.cc:83] Run] Reduce precision for [TensorScatterUpdate] input 1
